In [2]:
# Import required libraries
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
# Load data from a file
def load_data(file_path):
    with open(file_path, "r") as file:
        return file.readlines()

# Load datasets
train_file = "path/to/train.txt"
val_file = "path/to/emotion/val.txt"
test_file = "path/to/test.txt"
train = load_data(train_file)
val = load_data(val_file)
test = load_data(test_file)

In [4]:
# Split data into tweets and labels
def get_tweet(data):
    tweets = []
    labels = []
    for line in data:
        tweet, label = line.strip().split(';')
        tweets.append(tweet)
        labels.append(label)
    return tweets, labels

In [5]:
# Extract tweets and labels
tweets, labels = get_tweet(train)
val_tweets, val_labels = get_tweet(val)
test_tweets, test_labels = get_tweet(test)

In [6]:
# Tokenize tweets
def get_sequences(tokenizer, tweets, maxlen=50):
    sequences = tokenizer.texts_to_sequences(tweets)
    padded = pad_sequences(sequences, truncating='post', padding='post', maxlen=maxlen)
    return padded

In [7]:
# Tokenize tweets
tokenizer = Tokenizer(num_words=10000, oov_token='<UNK>')
tokenizer.fit_on_texts(tweets)
padded_train_seq = get_sequences(tokenizer, tweets)
padded_val_seq = get_sequences(tokenizer, val_tweets)
padded_test_seq = get_sequences(tokenizer, test_tweets)

In [8]:
# Create label index mappings
classes = set(labels)
class_to_index = {c: i for i, c in enumerate(classes)}
index_to_class = {i: c for c, i in class_to_index.items()}
names_to_ids = lambda labels: np.array([class_to_index.get(x) for x in labels])

In [9]:
# Convert labels to indices
train_labels = names_to_ids(labels)
val_labels = names_to_ids(val_labels)
test_labels = names_to_ids(test_labels)

In [10]:
# Build the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(10000, 16),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20)),
    tf.keras.layers.Dense(len(classes), activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
# Train the model
h = model.fit(padded_train_seq, train_labels, validation_data=(padded_val_seq, val_labels), epochs=20)

Epoch 1/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 35s 59ms/step - accuracy: 0.3673 - loss: 1.5477 - val_accuracy: 0.6850 - val_loss: 0.9055
Epoch 2/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 27s 53ms/step - accuracy: 0.7513 - loss: 0.6979 - val_accuracy: 0.7470 - val_loss: 0.6879
Epoch 3/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 22s 43ms/step - accuracy: 0.8590 - loss: 0.4187 - val_accuracy: 0.8300 - val_loss: 0.5177
Epoch 4/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 23s 47ms/step - accuracy: 0.9070 - loss: 0.2798 - val_accuracy: 0.8590 - val_loss: 0.4456
Epoch 5/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 27s 54ms/step - accuracy: 0.9419 - loss: 0.1876 - val_accuracy: 0.8765 - val_loss: 0.4242
Epoch 6/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 26s 53ms/step - accuracy: 0.9607 - loss: 0.1327 - val_accuracy: 0.8720 - val_loss: 0.4625
Epoch 7/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 29s 58ms/step - accuracy: 0.9684 - loss: 0.1075 - val_accuracy: 0.8810 - val_loss: 0.4296
Epoch 8/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 26s 51ms/step - accuracy: 0.9735 - loss: 0.0912 - 

In [18]:
# Function to predict emotion from the command line
def predict_emotion():
    while True:
        sentence = input("Enter a sentence (0 to quit): ")
        if sentence == "0":
            print("---------------------------------END--------------------------------------------")
            break
        if sentence == "":
            print("Enter the Sentence..!!")
            continue
        seq = tokenizer.texts_to_sequences([sentence])
        seq = pad_sequences(seq, maxlen=50, padding='post')
        p = model.predict(seq)[0]
        pred_class = index_to_class[np.argmax(p).astype('uint8')].capitalize()
        print(f"Predicted emotion: {pred_class}")
        print("--------------------------------------------------------------------------------")

In [19]:
# Start the prediction loop
predict_emotion()

Enter a sentence (0 to quit):  It was a wonderful day with NGO event.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Predicted emotion: Joy
--------------------------------------------------------------------------------


Enter a sentence (0 to quit):  I feel very low today.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Predicted emotion: Sadness
--------------------------------------------------------------------------------


Enter a sentence (0 to quit):  0


---------------------------------END--------------------------------------------
